# Scraping with Python

This is my notebook to learn how to web scrape with Python.

So here we go!


## Lesson 1

### Basic to web scraping:
- Lybraries we will use:
    - urllib
    - BeautifilSoup (bs4)

### How to Scrape:
1. Use urllib to make a request to the web page you want.
1. Parse the html to a BeautifilSoup object.
1. Find the data you need in the object.
1. You're done.
        

In [ ]:
# Here we have a simple exemple of the process:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://alura-site-scraping.herokuapp.com/hello-world.php"
response = urlopen(url) # Get on url
html = response.read() # Read the html data from response
soup = BeautifulSoup(html, 'html.parser') # Parse HTML to a Beautiful Soup OBJ
print(soup.find('h1', id='hello-world').get_text()) # Find the desired tag
print(soup.find('p').get_text())

## Lesson 2

### Improving a request:
Sometimes we need to improve a request to make it work with the page we need.
This can be adding a authorization or other headers or even a request body. To do this we create a Request and add things to this request, as in the code bellow.

### Cleaning up the data:

The object we get from response.read() is of type bytes. It's sometimes better to convert it to a str as strs have easier and greater options for a treatment.

It's always a good idea to decode and clean up the data we received to improve the convertion to a Beautiful Soup object.


In [ ]:
from urllib.request import Request, urlopen

url = 'https://www.alura.com.br'
headers = {'User-Agent': 'Chrome/76.0.3809.100'} # Creating a headers dictionary.

req = Request(url, headers = headers) # Creating the request using the headers dictionary.
response = urlopen(req)
b = response.read()

html = b.decode('utf-8') # Decoding bytes to str using UTF-8.
html = html.split() # Spliting the str in empty spaces, line breaks and tabs.
html = " ".join(html) # Re-joining the data with only 1 whitespace between everything.
html = html.replace('> <', '><') # Removing the whitespace between tags.

# For a better use let's create a function to do all this for us.

def clean_input(input):
    return " ".join(input.split()).replace('> <', '><')


## Lesson 3

### Working with bs4 BeautifulSoup:
- Parse a html string using the html.parser
- To improve visualization use soup.prettify()
- We can find tags using soup.find('tagname')
- Once on a tag we can find it's atributes with tag.attrs()



In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

def clean_input(input):
    return " ".join(input.split()).replace('> <', '><')

url = 'https://www.alura.com.br'
headers = {'User-Agent': 'Chrome/76.0.3809.100'} # Creating a headers dictionary.

req = Request(url, headers = headers) # Creating the request using the headers dictionary.
response = urlopen(req)
b = response.read()

html = clean_input(b.decode('utf-8')) # Decoding and cleaning.

soup = BeautifulSoup(html, 'html.parser') # Parsing.
soup.prettify() # Prettify

print(soup.find('h1')) # Find first h1
print(soup.find('h1').attrs) # Get atributes from first h1

## Lesson 4

### Find() and find_all()

You can use find() to find a single tag and find_all() for multiple tags acordingly to some filters.

- find(tag, attributes, recursive, text, **kwargs)
- find_all(tag, attributes, recursive, text, limit, **kwargs)

### Find_parent() and find_siblings()

Find parents and siblings are an easy way to navigate throught the html to get where you want to get

- find_parent(tag, attributes, text, **kwargs)
- find_parents(tag, attributes, text, limit **kwargs)
- find_next_sibling(tag, attributes, text, **kwargs)
- find_next_siblings(tag, attributes, text, limit **kwargs)
- find_previous_sibling(tag, attributes, text, **kwargs)
- find_previous_siblings(tag, attributes, text, limit **kwargs)


## Lesson 5

### Identifying and storing data.

Now we need to identify where in the html the data we need is and store it.

- Open the page in a web browser and analyse it.
- Figure out where the important data is.
- Store this data in a dictionary or a database.

### Working with Data Frames

Data Frames are usefull to convert your data to a file better suited for data analisys.

- We will use Pandas
- Create a dataframe from our dict and export it to a file.

### Storing images

Sometimes we are interested in storing images as well. To do this we use urlretrieve.

- Find the source (src) of the image.
- Use urlretrieve to download it to a directory.


In [ ]:
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

def clean_input(input):
    return " ".join(input.split()).replace('> <', '><')

url = 'https://alura-site-scraping.herokuapp.com/index.php'
headers = {'User-Agent': 'Chrome/76.0.3809.100'} # Creating a headers dictionary.

req = Request(url, headers = headers) # Creating the request using the headers dictionary.
response = urlopen(req)
b = response.read()

html = clean_input(b.decode('utf-8')) # Decoding and cleaning.

soup = BeautifulSoup(html, 'html.parser') # Parsing.

cards = [] # Array to hold all cards.
card = {} # Dictionary to store the information.

anuncio = soup.find('div', {'class': 'well card'}) # Find one add in the page.

infos = anuncio.find('div', {'class':'body-card'}).find_all('p') # Find the infos inside a body of card.

for info in infos:
    card[info.get('class')[0].split('-')[-1]] = info.get_text() # Storing infos to dict dynamicaly.

card['valor'] = anuncio.find('p',{'class':'txt-value'}).get_text() # Storing more info from other divs

items = anuncio.find('div', {'class':'body-card'}).ul.find_all('li') # Find the accessories inside a body of card.

items.pop() # Removing last item '...'

accessories = [] # Create array for accessories

for a in items :
        accessories.append(a.get_text().replace('► ', '')) # Adding acessories in array

card['accessories'] = accessories # Adding to dict

dataset = pd.DataFrame.from_dict(card, orient = 'index').T # Creating a data Frame from our card data.
dataset.to_csv('./output/data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig') # exporting the info.

image = anuncio.find('div', {'class':'image-card'}).find('img') # Find the image inside the card.
urlretrieve(image.get('src'), './output/img/' + image.get('src').split('/')[-1]) # Downloading the image to a data folder

## Lesson 6

### Scrapping the entire page

Apply what we learned on the previous lesson to all adds on the page.

In [13]:
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

def clean_input(input):
    return " ".join(input.split()).replace('> <', '><')

def parse_anuncio(anuncio):
    card = {} # Dictionary to store the information.

    infos = anuncio.find('div', {'class':'body-card'}).find_all('p') # Find the infos inside a body of card.

    for info in infos:
        card[info.get('class')[0].split('-')[-1]] = info.get_text() # Storing infos to dict dynamicaly.

    card['valor'] = anuncio.find('p',{'class':'txt-value'}).get_text() # Storing more info from other divs

    items = anuncio.find('div', {'class':'body-card'}).ul.find_all('li') # Find the accessories inside a body of card.
    items.pop() # Removing last item '...'
    accessories = [] # Create array for accessories

    for a in items :
            accessories.append(a.get_text().replace('► ', '')) # Adding acessories in array
    card['accessories'] = accessories # Adding to dict

    image = anuncio.find('div', {'class':'image-card'}).find('img') # Find the image inside the card.
    urlretrieve(image.get('src'), './output/img/' + image.get('src').split('/')[-1]) # Downloading the image to a data folder

    return card

def scrape_page():
    url = 'https://alura-site-scraping.herokuapp.com/index.php'
    headers = {'User-Agent': 'Chrome/76.0.3809.100'} # Creating a headers dictionary.
    req = Request(url, headers = headers) # Creating the request using the headers dictionary.
    response = urlopen(req)
    b = response.read()
    html = clean_input(b.decode('utf-8')) # Decoding and cleaning.
    soup = BeautifulSoup(html, 'html.parser') # Parsing.

    cards = [] # Array to hold all cards.

    anuncios = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card") # Find all adds in page 

    for anuncio in anuncios: # Loop through adds
        cards.append(parse_anuncio(anuncio)) # Parse and add to list.

    dataset = pd.DataFrame(cards) # Creating a data Frame from our card data.
    dataset.to_csv('./output/data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig') # exporting the info.

    return dataset

print(scrape_page())

                     name category            motor           description  \
0   LAMBORGHINI AVENTADOR    USADO    Motor 1.8 16v  Ano 1993 - 55.286 km   
1                  BMW M2    USADO    Motor 3.0 32v  Ano 2018 - 83.447 km   
2                    ALFA    USADO    Motor 1.8 16v  Ano 2004 - 19.722 km   
3                   PUECH    USADO  Motor Diesel V8  Ano 1992 - 34.335 km   
4  LAMBORGHINI MURCIELAGO    USADO     Motor 1.0 8v     Ano 1991 - 464 km   
5            ASTON MARTIN    USADO  Motor Diesel V6  Ano 2004 - 50.189 km   
6                     TVR    USADO  Motor 4.0 Turbo  Ano 2014 - 17.778 km   
7               EXCALIBUR    USADO    Motor 3.0 32v  Ano 2009 - 81.251 km   
8                 MCLAREN     NOVO     Motor Diesel       Ano 2019 - 0 km   
9                  TOYOTA    USADO  Motor 4.0 Turbo  Ano 1999 - 12.536 km   

              location       valor  \
0  Belo Horizonte - MG  R$ 338.000   
1  Belo Horizonte - MG  R$ 346.000   
2  Rio de Janeiro - RJ  R$ 480.000   


## Lesson 7

### Scrapping the entire site.

Apply what we learned previously to scrape the website and all it's pages.

In [24]:
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

def clean_input(input):
    return " ".join(input.split()).replace('> <', '><')

def parse_anuncio(anuncio):
    card = {} # Dictionary to store the information.

    infos = anuncio.find('div', {'class':'body-card'}).find_all('p') # Find the infos inside a body of card.

    for info in infos:
        card[info.get('class')[0].split('-')[-1]] = info.get_text() # Storing infos to dict dynamicaly.

    card['valor'] = anuncio.find('p',{'class':'txt-value'}).get_text() # Storing more info from other divs

    items = anuncio.find('div', {'class':'body-card'}).ul.find_all('li') # Find the accessories inside a body of card.
    items.pop() # Removing last item '...'
    accessories = [] # Create array for accessories

    for a in items :
            accessories.append(a.get_text().replace('► ', '')) # Adding acessories in array
    card['accessories'] = accessories # Adding to dict

    # image = anuncio.find('div', {'class':'image-card'}).find('img') # Find the image inside the card.
    # urlretrieve(image.get('src'), './output/img/' + image.get('src').split('/')[-1]) # Downloading the image to a data folder

    return card

def scrape_page(page=1):
    url = 'https://alura-site-scraping.herokuapp.com/index.php?page=' + str(page)
    headers = {'User-Agent': 'Chrome/76.0.3809.100'} # Creating a headers dictionary.
    req = Request(url, headers = headers) # Creating the request using the headers dictionary.
    response = urlopen(req)
    b = response.read()
    html = clean_input(b.decode('utf-8')) # Decoding and cleaning.
    soup = BeautifulSoup(html, 'html.parser') # Parsing.

    cards = [] # Array to hold all cards.

    anuncios = soup.find('div', {"id": "container-cards"}).findAll('div', class_="card") # Find all adds in page 

    for anuncio in anuncios: # Loop through adds
        cards.append(parse_anuncio(anuncio)) # Parse and add to list.

    return cards

def scrape_site():
    url = 'https://alura-site-scraping.herokuapp.com/index.php'
    headers = {'User-Agent': 'Chrome/76.0.3809.100'} # Creating a headers dictionary.
    req = Request(url, headers = headers) # Creating the request using the headers dictionary.
    response = urlopen(req)
    b = response.read()
    html = clean_input(b.decode('utf-8')) # Decoding and cleaning.
    soup = BeautifulSoup(html, 'html.parser') # Parsing.
    total_pages = int(soup.find('span', class_='info-pages').get_text().split()[-1]) # Find total pages for search

    all_cards = []
    for page in range(1,total_pages+1):
        all_cards.extend(scrape_page(page))

    dataset = pd.DataFrame(all_cards) # Creating a data Frame from our card data.
    dataset.to_csv('./output/data/dataset.csv', sep=';', index = False, encoding = 'utf-8-sig') # exporting the info.

    return dataset

print(scrape_site())


                       name category            motor            description  \
0     LAMBORGHINI AVENTADOR    USADO    Motor 1.8 16v   Ano 1993 - 55.286 km   
1                    BMW M2    USADO    Motor 3.0 32v   Ano 2018 - 83.447 km   
2                      ALFA    USADO    Motor 1.8 16v   Ano 2004 - 19.722 km   
3                     PUECH    USADO  Motor Diesel V8   Ano 1992 - 34.335 km   
4    LAMBORGHINI MURCIELAGO    USADO     Motor 1.0 8v      Ano 1991 - 464 km   
..                      ...      ...              ...                    ...   
241           SUV REAR TIRE    USADO    Motor 3.0 32v   Ano 1998 - 74.292 km   
242                 ANTIQUE     NOVO    Motor 2.0 16v        Ano 2019 - 0 km   
243                   SPORT    USADO    Motor 2.0 16v  Ano 2001 - 102.776 km   
244                IMPERIAL    USADO    Motor 1.8 16v  Ano 2011 - 101.787 km   
245          KIA SPORTS CAR    USADO    Motor 3.0 32v   Ano 2001 - 88.564 km   

                location       valor  \